### Итоговое задание модуля Python-11

Датасет представляет собой таблицу с информацией о 300 тысячах поездок за первые пять дней сентября 2018 года и включает в себя следующую информацию:

- starttime — время начала поездки (дата, время);
- stoptime — время окончания поездки (дата, время);
- start station id — идентификатор стартовой стоянки;
- start station name — название стартовой стоянки;
- start station latitude, start station longitude — географическая широта и долгота стартовой стоянки;
- end station id — идентификатор конечной стоянки;
- end station name — название конечной стоянки;
- end station latitude, end station longitude — географическая широта и долгота конечной стоянки;
- bikeid — идентификатор велосипеда;
- usertype — тип пользователя (Customer — клиент с подпиской на 24 часа или на три дня, Subscriber — подписчик с годовой арендой велосипеда);
- birth year — год рождения клиента;
- gender — пол клиента (0 — неизвестный, 1 — мужчина, 2 — женщина).

In [1]:
import pandas as pd
sport_data = pd.read_csv('./data/citibike-tripdata.csv')

In [3]:
# копируем в новый df, чтобы не подгружать файл, если чего не так пойдет
sport_df = sport_data.copy()

In [4]:
# 6.1
# Сколько пропусков в столбце start station id?
sport_df['start station id'].isna().sum()

169

In [5]:
# 6.2
# Какой тип данных имеют столбцы starttime и stoptime?
sport_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 300000 entries, 0 to 299999
Data columns (total 14 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   starttime                300000 non-null  object 
 1   stoptime                 300000 non-null  object 
 2   start station id         299831 non-null  float64
 3   start station name       299831 non-null  object 
 4   start station latitude   300000 non-null  float64
 5   start station longitude  300000 non-null  float64
 6   end station id           299831 non-null  float64
 7   end station name         299831 non-null  object 
 8   end station latitude     300000 non-null  float64
 9   end station longitude    300000 non-null  float64
 10  bikeid                   300000 non-null  int64  
 11  usertype                 300000 non-null  object 
 12  birth year               300000 non-null  int64  
 13  gender                   300000 non-null  int64  
dtypes: f

In [6]:
# 6.3
# Найдите идентификатор самой популярной стартовой стоянки. Запишите идентификатор в виде целого числа.
sport_df['start station id'].value_counts().index[0]
sport_df['start station id'].mode()[0]

281.0

In [7]:
# 6.4
# Велосипед с каким идентификатором является самым популярным?
sport_df['bikeid'].value_counts().index[0]

33887

In [10]:
# 6.5
# Какой тип клиентов (столбец usertype) является преобладающим — Subscriber или Customer? 
# В качестве ответа запишите долю клиентов преобладающего типа среди общего количества клиентов. 
# Ответ округлите до сотых.
print(sport_df.usertype.value_counts().index[0])
sport_df.usertype.value_counts()[0] / sport_df.shape[0]

Subscriber


0.7740066666666666

In [15]:
# 6.6
# Кто больше занимается велоспортом — мужчины или женщины? В ответ запишите число поездок для той группы, у которой их больше.
print(['unknown', 'men', 'women'][sport_df.gender.value_counts().index[0]])
sport_df.gender.value_counts().iloc[0]

men


183582

In [25]:
# 6.7
# Выберите утверждения,которые соответствуют нашим данным:
# A. Число уникальных стартовых и конечных стоянок, которыми воспользовались клиенты, не равны друг другу
# B. В рассматриваемые дни минимальный возраст клиента составлял 10 лет
# C. Самой непопулярной стартовой стоянкой из тех, которыми воспользовались клиенты, является стоянка с названием "Eastern Pkwy & Washington Ave"
# D. Наибольшее количество поездок завершается на стоянке под названием "Liberty Light Rail"

sport_df['start station id'].nunique() != sport_df['end station id'].nunique()             # A. True
(pd.to_datetime(sport_df['starttime']).dt.year - sport_df['birth year']).min() == 10       # B. False
sport_df['start station name'].value_counts().index[-1] == 'Eastern Pkwy & Washington Ave' # C. True
sport_df['end station name'].value_counts().index[0] == 'Liberty Light Rail'               # D. False

False

In [28]:
# 6.8
# В первую очередь удалим лишнюю информацию из данных.
# В наших данных присутствуют столбцы, которые дублируют информацию друг о друге: 
# это столбцы с идентификатором и названием стартовой и конечной стоянки. 
# Удалите признаки идентификаторов стоянок. Сколько столбцов осталось?
sport_df.drop(['start station id', 'end station id'], inplace=True, axis=1)
len(sport_df.columns)

12

In [31]:
# 6.9
# Замените признак birth year на более понятный признак возраста клиента age. 
# Годом отсчёта возраста выберите 2018 год. Столбец birth year удалите из таблицы. 
# Сколько поездок совершено клиентами старше 60 лет?
sport_df['age'] = 2018 - sport_df['birth year']
sport_df.drop(['birth year'], inplace=True, axis=1)
sport_df[sport_df.age > 60].shape[0]

11837

In [50]:
# 6.10
# Создайте признак длительности поездки trip duration. 
# Для этого вычислите интервал времени между временем окончания поездки (stoptime) и временем её начала (starttime) в секундах. 
# Рассчитайте среднее значение по новому столбцу — среднюю длительность поездки, а затем переведите её в секунды. 
# Ответ округлите до целого.
# Для того чтобы преобразовать временной интервал timedelta в секунды, 
# используется атрибут seconds (в datetime атрибут был second — в единственном числе).
sport_df['trip duration'] = (pd.to_datetime(sport_df['stoptime']) - pd.to_datetime(sport_df['starttime'])).dt.total_seconds()
print(sport_df['trip duration'].mean())

sport_df['trip duration'] = pd.to_datetime(sport_df['stoptime']) - pd.to_datetime(sport_df['starttime'])
print(sport_df['trip duration'].mean().seconds)

1126.5588499066666
1126


In [60]:
# 6.11
# Создайте «признак-мигалку» weekend, который равен 1, если поездка начиналась в выходной день (суббота или воскресенье), 
# и 0 — в противном случае. Выясните, сколько поездок начиналось в выходные.
def is_weekend(dt):
    return dt.dayofweek > 4
sport_df['weekend'] = pd.to_datetime(sport_df.starttime).apply(is_weekend)
sport_df.weekend.sum()

115135

In [69]:
# 6.12
# Создайте признак времени суток поездки time_of_day. Время суток будем определять из часа начала поездки. Условимся, что:
# - поездка совершается ночью (night), если её час приходится на интервал от 0 (включительно) до 6 (включительно) часов;
# - поездка совершается утром (morning), если её час приходится на интервал от 6 (не включительно) до 12 (включительно) часов;
# - поездка совершается днём (day), если её час приходится на интервал от 12 (не включительно) до 18 (включительно) часов;
# - поездка совершается вечером (evening), если её час приходится на интервал от 18 (не включительно) до 23 часов (включительно).
# Во сколько раз количество поездок, совершённых днём, больше, чем количество поездок, совёршенных ночью, 
# за представленный в данных период времени? Ответ округлите до целых.
def how_time_of_day(datetime):
    match datetime.time().hour:
        case x if x < 7:
            return 'night'
        case x if x < 13:
            return 'morning'
        case x if x < 19:
            return 'day'
        case _:
            return 'evening'
sport_df['time_of_day'] = pd.to_datetime(sport_df.starttime).apply(how_time_of_day)
int(round(sport_df.time_of_day.value_counts()['day'] / sport_df.time_of_day.value_counts()['night']))

9